In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

# Langsmith Tracing
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [8]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from pprint import pprint

llm = ChatGroq(model='groq/compound-mini')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000136DB6AB500>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000136DB6AAF60>, model_name='groq/compound-mini', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
res = llm.invoke('Can you explain me about rules of f1 in 10 lines').content
pprint(res)

('Here are the rules of F1 in 10 lines:\n'
 '\n'
 '1. The Formula 1 World Championship consists of multiple Grands Prix races.\n'
 '2. Each team has two drivers competing in each race.\n'
 '3. The objective is to complete the most laps in the shortest time.\n'
 '4. Drivers start the race from a grid position determined by qualifying '
 'laps.\n'
 '5. The driver with the fastest lap in qualifying gets pole position.\n'
 '6. Drivers can make pit stops to change tires and refuel (though refueling '
 'is not allowed in some sessions).\n'
 '7. Each driver has a limited number of engine components that can be used '
 'throughout the season.\n'
 '8. Drivers must complete a minimum of 90% of the laps to be classified as '
 'finishers.\n'
 '9. Safety cars and Virtual Safety Cars (VSC) can be deployed in case of '
 'accidents or hazardous conditions.\n'
 '10. Points are awarded to the top 10 finishers (25, 18, 15, 12, 10, 8, 6, 4, '
 '2, 1) to determine the championship standings.')


### HumanMessages & System Messages

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
                SystemMessage(content="Translate the user message to French"),
                HumanMessage(content="You look beautiful tonight!")
]

response = llm.invoke(messages)
response.content

"Tu as l'air magnifique ce soir !"

In [13]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()
str_parser.invoke(response)

"Tu as l'air magnifique ce soir !"

In [14]:
# Using LCEL For Chaining

chain1 = llm|str_parser
chain1.invoke(messages)

"You look beautiful tonight translates to:\n\nTu as l'air magnifique ce soir!\n\nor, in a more formal tone:\n\nVous êtes magnifique ce soir!"

In [20]:
from langchain_core.prompts import ChatPromptTemplate

gen_prompt  = "Translate the following into {language}"
prompt = ChatPromptTemplate.from_messages([
                ('system',gen_prompt),
                ("user", "{text}")
])

prompt.invoke({"language":"French","text":"Rahul is a really amazing person"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Rahul is a really amazing person', additional_kwargs={}, response_metadata={})])

In [21]:
chain2  = prompt|llm|str_parser
chain2.invoke({"language":"French","text":"Rahul is a really amazing person"})

'To translate the sentence "Rahul is a really amazing person" into French, I will use my language capabilities.\n\nThe translation is: "Rahul est une personne vraiment incroyable."\n\nHere\'s a breakdown of the translation:\n\n- "Rahul" remains the same as it\'s a proper noun.\n- "is" is translated to "est".\n- "a" is translated to "une".\n- "really" is translated to "vraiment".\n- "amazing" is translated to "incroyable".\n- "person" is translated to "personne".'